In [3]:
from nupack import *
print('nupack imported')

nupack imported


# Trial Switch Generation

In [5]:
# Define Model Space
my_model = Model(material='rna', celsius=37)

# Define Domains
dA = Domain('N11', name='dA')
dB = Domain('N25', name='dB')
dG = Domain('GGG', name='dG')
dS = Domain('GGACTTTAGAACAGAGGAGATAAAGATG', name='dS')
dL = Domain('NAACCTGGCGGCAGCGCAAAAG', name='dL')

# Define Strands
sSwitch = TargetStrand([dG, ~dB, ~dA, dS, dA, dL], name='sSwitch')
sTrigger = TargetStrand([dA, dB], name='sTrigger')

# Define Complexes
cSwitch = TargetComplex([sSwitch], '............................(((((((((((...(((((............)))))...)))))))))))......................', name='cSwitch')
cTrigger = TargetComplex([sTrigger], 'U36', name='cTrigger')
cActivated = TargetComplex([sTrigger, sSwitch], '((((((((((((((((((((((((((((((((((((+...))))))))))))))))))))))))))))))))))))...(((((............)))))....................................', name='cActivated')

# Define Tubes
reactants = TargetTube(on_targets={cSwitch: 1e-08, cTrigger: 1e-08}, 
                       off_targets=SetSpec(max_size=2, exclude=[cActivated]), name='Reactants')
products = TargetTube(on_targets={cActivated: 1e-08}, 
                      off_targets=SetSpec(max_size=2), name='Products')
crosstalk = TargetTube(on_targets={cSwitch: 1e-08, cTrigger: 1e-08}, 
                       off_targets=SetSpec(max_size=2, exclude=[cActivated, [sSwitch, sTrigger]]), name='Crosstalk')

# Define Constraints
Target_Sequence = 'GAUGAGCACUUUUAAAGUUCUGCUAUGUGGCGCGGUAUUAUCCCGUAUUGACGCCGGGCAAGAGCAACUCGGUCGCCGCAUACACUAUUCUCAGAAUGACUUGGUUGAGUACUCACCAGUCACAGAAAAGCAUCUUACGGAUGGCAUGACAGUAAGAGAAUUAUGCAGUGCUGCCAUAACCAUGAGUGAUAACACUGCGGCCAACUUACUUCUGACAACGAUCGGAGGACCGAAGGAGCUAACCGCUUUUUUGCACAACAUGGGGGAUCAUGUAACUCGCCUUGAUCGUUGGGAACCGGAGCUGAAUGAAGCCAUACCAAACGACGAGCGUGACACCACGAUGCCUGUAGCAAUGGCAACAACGUUGCGCAAACUAUUAACUGGCGAACUACUUACUCUAGCUUCCCGGCAACAAUUAAUAGACUGGAUGGAGGuGGAUAAAGUUGCAGGACCACUUCUGCGCUCGGCCCUUCCGGCUGGCUGGUUUAUUGCUGAUAAAUCUGGAGCCGGUGAGCGUGGcaGUCGCGGUAUCAUUGCAGCACUGGGGCCAGAUGGUAAGCCCUCCCGUAUCGUAGUUAUCUACACGACGGGGAGUCAGGCAACUAUGGAUGAACGAAAUAGACAGAUCGCUGAGAUAGGUGCCUCACUGAUUAAGCAUUG'
window = Window(sTrigger, sources=[Target_Sequence])

diversity = Diversity(word=4, types=2, scope=sTrigger)

# Run Trials
my_options = DesignOptions(f_stop=0.02, seed=93)
my_design = tube_design(tubes=[reactants, products, crosstalk], hard_constraints=[window, diversity], model=my_model, options=my_options)
my_results = my_design.run(trials=1)[0]

#Display Results
my_results

Domain,Sequence
dA,GGCAACAATTA
dA*,TAATTGTTGCC
dB,ATAGACTGGATGGAGGTGGATAAAG
dB*,CTTTATCCACCTCCATCCAGTCTAT
dG,GGG
dL,CAACCTGGCGGCAGCGCAAAAG
dS,GGACTTTAGAACAGAGGAGATAAAGATG
Strand,Sequence
sSwitch,GGGCTTTATCCACCTCCATCCAGTCTATTAATTGTTGCCGGACTTTAGAACAGAGGAGATAAAGATGGGCAACAATTACAACCTGGCGGCAGCGCAAAAG
sTrigger,GGCAACAATTAATAGACTGGATGGAGGTGGATAAAG


# Multiple Generated Switches

In [9]:
# Define Model Space
my_model = Model(material='rna', celsius=37)

# Define Number of Switches and Initialize Working Variables
systems = 5
crosstalk_targets = {}
crosstalk_excludes = []
step_tubes = []

window_targets = []
repeat_targets = []


# Loop over orthogonal systems

for i in range(systems):

    # Define Domains
    dA = Domain('N11', name=['dA', i])
    dB = Domain('N25', name=['dB', i])
    dG = Domain('GGG', name=['dG', i])
    dS = Domain('GGACTTTAGAACAGAGGAGATAAAGATG', name=['dS', i])
    dL = Domain('NAACCTGGCGGCAGCGCAAAAG', name=['dL', i])

    # Define Strands
    sSwitch = TargetStrand([dG, ~dB, ~dA, dS, dA, dL], name=['sSwitch', i])
    sTrigger = TargetStrand([dA, dB], name=['sTrigger', i])

    # Define Complexes
    cSwitch = TargetComplex([sSwitch], '............................(((((((((((...(((((............)))))...)))))))))))......................', name=['cSwitch', i])
    cTrigger = TargetComplex([sTrigger], 'U36', name=['cTrigger', i])
    cActivated = TargetComplex([sTrigger, sSwitch], '((((((((((((((((((((((((((((((((((((+...))))))))))))))))))))))))))))))))))))...(((((............)))))....................................', name=['cActivated', i])

    # Define Step Tubes
    reactants = TargetTube(on_targets={cSwitch: 1e-08, cTrigger: 1e-08}, 
                       off_targets=SetSpec(max_size=2, exclude=[cActivated]), name=['Reactants', i])
    products = TargetTube(on_targets={cActivated: 1e-08}, 
                      off_targets=SetSpec(max_size=2), name= ['Products', i])
    step_tubes += [reactants, products]
    
    # Define Global Crosstalk Variables
    crosstalk_targets.update({cSwitch: 1e-08, cTrigger: 1e-08})
    crosstalk_excludes += [cActivated, [sSwitch, sTrigger]]
    
    # Define Constraint Targets
    window_targets += [dA, dB]
    repeat_targets += [dA, dB]

# Define Global Crosstalk Tubes
crosstalk_tube = TargetTube(on_targets=crosstalk_targets, 
                        off_targets=SetSpec(max_size=2, exclude=crosstalk_excludes), name='Crosstalk')

# Define tube set
my_tubes = step_tubes + [crosstalk_tube]

# Define Constraints
window0 = Window(window_targets[0:2], sources=[Target_Sequence])
window1 = Window(window_targets[2:4], sources=[Target_Sequence])
window2 = Window(window_targets[4:6], sources=[Target_Sequence])
window3 = Window(window_targets[6:8], sources=[Target_Sequence])
window4 = Window(window_targets[8:10], sources=[Target_Sequence])
repeat = Diversity(word=4, types=2, scope=repeat_targets)

# Define defect weight of N for global crosstalk tube (maintain balance between elementary step tubes and global crosstalk tube as number of orthogonal systems increases) 
my_weights = Weights(my_tubes)
my_weights[:,:,:,crosstalk_tube] *= systems

# Run Trials
my_options = DesignOptions(f_stop=0.10, seed=93)
my_design = tube_design(tubes=my_tubes, defect_weights=my_weights, hard_constraints=[window0, window1, window2, window3, window4, repeat], model=my_model, options=my_options)
my_jobs = my_design.launch(trials=1, checkpoint='my_checkpoints', interval=60)

Starting design 0 from scratch with checkpoints in my_checkpoints/0


In [8]:
my_current_results = my_jobs.current_results()
my_current_results[0]

Domain,Sequence
dA[0],GGGCCAGATGG
dA[0]*,CCATCTGGCCC
dA[1],GAGTCAGGCAA
dA[1]*,TTGCCTGACTC
dA[2],GAACTACTTAC
dA[2]*,GTAAGTAGTTC
dA[3],AAGAGCAACTC
dA[3]*,GAGTTGCTCTT
dA[4],GGAACCGGAGC
dA[4]*,GCTCCGGTTCC
